### Задание 1

In [8]:
import pandas as pd
import zipfile
import json
import sqlite3
from pathlib import Path

In [2]:
# Читаем файл json и сохраняем в DataFrame
okved_df = pd.read_json('okved_2.json.zip')

In [3]:
okved_df

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,
...,...,...,...,...,...
2813,Q,,Q,ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ ЗДРАВООХРАНЕНИЯ И СОЦИА...,Этот раздел включает:\n- предоставление деятел...
2814,R,,R,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",Данный раздел включает:\n- широкий спектр видо...
2815,S,,S,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,Этот раздел включает:\n- деятельность обществе...
2816,T,,T,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,


In [4]:
okved_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2818 entries, 0 to 2817
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         2818 non-null   object
 1   parent_code  2818 non-null   object
 2   section      2818 non-null   object
 3   name         2818 non-null   object
 4   comment      2818 non-null   object
dtypes: object(5)
memory usage: 110.2+ KB


In [5]:
# Создаём БД hw1.db и таблицу okved в ней
create_stmt = \
"""
create table if not exists okved(
    code text,
    parent_code text,
    section text,
    name text,
    comment text
)
"""
with sqlite3.connect('hw1.db') as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(create_stmt)
    except(Exception) as error:
        print(error)
        connection.rollback()
    else:
        connection.commit()
    finally:
        cursor.close()

In [6]:
# Записываем данные из DataFrame в БД
with sqlite3.connect('hw1.db') as connection:
    okved_df.to_sql('okved', connection, if_exists='append', index=False, method='multi')

In [7]:
# Проверяем получившийся результат
with sqlite3.connect('hw1.db') as connection:
    df = pd.read_sql('select * from okved', connection)

display(df)

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,
...,...,...,...,...,...
2813,Q,,Q,ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ ЗДРАВООХРАНЕНИЯ И СОЦИА...,Этот раздел включает:\n- предоставление деятел...
2814,R,,R,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",Данный раздел включает:\n- широкий спектр видо...
2815,S,,S,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,Этот раздел включает:\n- деятельность обществе...
2816,T,,T,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,


In [10]:
df[df.name.str.lower() == 'деятельность в сфере телекоммуникаций']

,code,parent_code,section,name,comment
2193,61,J,J,Деятельность в сфере телекоммуникаций,Эта группировка включает:\n- деятельность по п...


### Задание 2

In [1]:
import pandas as pd
import zipfile
import sqlite3
from pathlib import Path
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from multiprocessing.pool import ThreadPool as Pool
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

In [2]:
from concurrent.futures import wait
from typing import Tuple
from zipfile import ZipFile


file_path = Path('C:', 'Users', 'greym', 'Downloads', 'hw1', 'egrul.json.zip')
NPROC = os.cpu_count() - 1  # количество процессоров
MAX_WORKERS = max(1, round(os.cpu_count() / 2))  # количество потоков


def unpacker(path: 'str | Path') -> pd.DataFrame:
    """Распаковщик данных из zip-архива и парсер JSON"""

    def parser_json(zipobj: ZipFile, json_file: str) -> Tuple[str, pd.DataFrame]:
        """Парсит один файл JSON"""
        with zipobj.open(json_file) as file:
            return (json_file, pd.read_json(file))

    with zipfile.ZipFile(path, 'r') as zipobj:
        files = zipobj.infolist()
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            # загружаем и отмечаем каждый будущий результат своим файлом
            counter = 0
            while files:
                futures = []
                file_lasts = len(files)
                for i in range(MAX_WORKERS):
                    if i < file_lasts:
                        futures.append(executor.submit(parser_json, zipobj, files[i].filename))

                files = files[MAX_WORKERS:]

                success_futures, _ = wait(futures)
                for future in success_futures:
                    filename, df = future.result()
                    counter += 1
                    print(f"File {filename} loaded ({counter})")
                    yield df


def parser_data(value: dict) -> dict:
    """Парсер атрибута data"""
    # Получаем значения КодОКВЭД и НаимОКВЭД. Если нет, устанавливаем значения в None
    main_val = dict(code_okved=value.get('СвОКВЭД', {}).get('СвОКВЭДОсн', {}).get('КодОКВЭД', None),
                    name_okved=value.get('СвОКВЭД', {}).get('СвОКВЭДОсн', {}).get('НаимОКВЭД', None),
                    type_okved='Осн')
    # add_val = value.get('СвОКВЭД', {}).get('СвОКВЭДДоп', def_dct)
    # if not isinstance(add_val, list):
    #    add_val = [add_val]
    # Фильтруем по 61 ОКВЭД
    code = main_val['code_okved']
    return main_val if code and code.startswith('61') else None


def processor_df(df: pd.DataFrame) -> pd.DataFrame:
    """Парсер и обработчик данных"""
    # Парсим атрибут data и записываем данные в общий DF
    df = df.assign(data=df.data.apply(parser_data))
    # Исключаем пустые данные (т.е. те, что не попали под ОКВЭД 61)
    df = df[df.data.notna()]
    # Разбиваем атрибут data на столбцы, и добавляем их в общий DF
    cols = ['ogrn', 'inn', 'kpp', 'name', 'full_name']
    return df[cols].join(df.data.apply(pd.Series))


# Создаём генератор-парсер данных из .zip архива в формате JSON
gen_dfs = unpacker(file_path)

In [ ]:
# Парсим данные из JSON файлов в DF в параллельном режиме
with Pool(processes=NPROC) as process_pool:
    dfs = process_pool.map(processor_df, gen_dfs, chunksize=NPROC)

File 06439.json loaded (1)
File 04669.json loaded (2)
File 03534.json loaded (3)
File 00393.json loaded (4)
File 10339.json loaded (5)
File 07388.json loaded (6)
File 09162.json loaded (7)
File 05575.json loaded (8)
File 08312.json loaded (9)
File 06849.json loaded (10)
File 05296.json loaded (11)
File 08667.json loaded (12)
File 10549.json loaded (13)
File 10851.json loaded (14)
File 03216.json loaded (15)
File 07553.json loaded (16)
File 10245.json loaded (17)
File 08935.json loaded (18)
File 07602.json loaded (19)
File 00981.json loaded (20)
File 07293.json loaded (21)
File 08377.json loaded (22)
File 08450.json loaded (23)
File 10193.json loaded (24)
File 03937.json loaded (25)
File 06832.json loaded (26)
File 10364.json loaded (27)
File 05046.json loaded (28)
File 06268.json loaded (29)
File 02313.json loaded (30)
File 04600.json loaded (31)
File 03075.json loaded (32)
File 06266.json loaded (33)
File 05820.json loaded (34)
File 11118.json loaded (35)
File 04874.json loaded (36)
F

In [ ]:
# Объединяем все данные по всем юр. лицам в один DF
egrul_df = pd.concat(dfs, ignore_index=True)

File 04669.json loaded
File 06439.json loaded
File 03534.json loaded
File 00393.json loaded
File 10339.json loaded
File 07388.json loaded
File 09162.json loaded
File 05575.json loaded
File 08312.json loaded
File 06849.json loaded
File 08667.json loaded
File 05296.json loaded
File 10549.json loaded
File 10851.json loaded
File 07553.json loaded
File 03216.json loaded
File 08935.json loaded
File 10245.json loaded
File 07602.json loaded
File 00981.json loaded
File 08377.json loaded
File 07293.json loaded
File 10193.json loaded
File 08450.json loaded
File 06832.json loaded
File 03937.json loaded
File 05046.json loaded
File 10364.json loaded
File 06268.json loaded
File 02313.json loaded
File 03075.json loaded
File 04600.json loaded
File 06266.json loaded
File 05820.json loaded
File 11118.json loaded
File 04874.json loaded
File 08771.json loaded
File 10194.json loaded
File 00407.json loaded
File 03811.json loaded
File 08657.json loaded
File 10723.json loaded
File 06595.json loaded
File 03702.

In [22]:
# Создаём таблицу telecom_companies БД hw1.db
create_stmt = \
"""
create table if not exists telecom_companiesokved(
    inn integer,
    ogrn integer,
    kpp integer,
    name text,
    full_name text,
    code_okved text,
    name_okved text,
    type_okved text
)
"""
with sqlite3.connect('hw1.db') as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(create_stmt)
    except(Exception) as error:
        print(error)
        connection.rollback()
    else:
        connection.commit()
    finally:
        cursor.close()

In [24]:
# Записываем данные из DataFrame в БД
rs = egrul_df.drop(['data'], axis=1)
print(rs)
with sqlite3.connect('hw1.db') as connection:
    rs.to_sql('telecom_companiesokved', connection,
                    if_exists='append',
                    index=False,
                    method='multi')

             ogrn         inn        kpp  \
0   1020201686130   257004913   25701001   
1   1020201257669   242002085   24201001   
2   1020201993712   266015083   26601001   
3   1020201775934   261010099   26101001   
4   1020202000169   266019271   26601001   
5   1020202000862   266021577   26601001   
6   1020201848325   263003883   26301001   
7   1020201932068   265011047   26501001   
8   1020100695438  0105037128  010501001   
9   1020100706856  0105038315  057301001   
10  1020100824743  0106004421  010701001   
11  1020100699970   105031366  344401001   
12  1020100704249   105036163   10501001   
13  1020100705460   105034952   10501001   

                                               name  \
0                                         ООО "РИК"   
1                                          ПК "УКВ"   
2                          ООО " АГЕНТСТВО "СИМПЭК"   
3                                         ООО "РИО"   
4                                ООО ИКФ "ИНТЕРКОМ"   
5        

In [25]:
# Проверяем получившийся результат
with sqlite3.connect('hw1.db') as connection:
    df = pd.read_sql('select * from telecom_companiesokved', connection)

display(df)

,inn,ogrn,kpp,name,full_name,code_okved,name_okved,type_okved
0,257004913,1020201686130,25701001,"ООО ""РИК""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РИК""",61.10.9,Деятельность в области связи на базе проводных...,Осн
1,242002085,1020201257669,24201001,"ПК ""УКВ""","ПРОИЗВОДСТВЕННЫЙ КООПЕРАТИВ ""УКВ""",61.10,Деятельность в области связи на базе проводных...,Осн
2,266015083,1020201993712,26601001,"ООО "" АГЕНТСТВО ""СИМПЭК""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АГЕН...",61.10,Деятельность в области связи на базе проводных...,Осн
3,261010099,1020201775934,26101001,"ООО ""РИО""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РИО""",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
4,266019271,1020202000169,26601001,"ООО ИКФ ""ИНТЕРКОМ""",ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ИНЖЕН...,61.10,Деятельность в области связи на базе проводных...,Осн
5,266021577,1020202000862,26601001,"ООО ""СИСТЕМЫ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СИСТ...",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
6,263003883,1020201848325,26301001,"ООО ""СВЯЗЬ ТРАНЗИТ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СВЯЗ...",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
7,265011047,1020201932068,26501001,"ООО ""КОРОНА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОРОНА""",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
8,105037128,1020100695438,10501001,"ООО ""АЭРОЛУЧ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АЭРО...",61.10,Деятельность в области связи на базе проводных...,Осн
9,105038315,1020100706856,57301001,ФКУ ЦИТОВ УФСИН РОССИИ ПО РЕСПУБЛИКЕ ДАГЕСТАН,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЦЕНТР ИНЖЕНЕР...",61.10.1,Деятельность по предоставлению услуг телефонно...,Осн
